---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [38]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [6]:
def graph_identification():
    
    # Your Code Here
    
    methods = []
    for G in P1_Graphs:
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
        if len(histogram) > 10:
            methods.append('PA')
        elif nx.average_clustering(G) < 0.1:
            methods.append('SW_H')
        else:
            methods.append('SW_L')
    
    return methods # Your Answer Here
                                                                                            
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [62]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [24]:
def salary_predictions():
    
    # Your Code Here
    
    from sklearn.ensemble import GradientBoostingClassifier
    
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    
    train = df[~df['ManagementSalary'].isnull()]
    test = df[df['ManagementSalary'].isnull()]
    
    X_train = train[['Department', 'clustering', 'degree', 'degree_centrality', 'closeness', 'betweeness', 'pr']]
    X_test = test[['Department', 'clustering', 'degree', 'degree_centrality', 'closeness', 'betweeness', 'pr']]
    y_train = train['ManagementSalary']
    
    clf = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
    test_pred = clf.predict_proba(X_test)[:, 1]
    
    results = pd.Series(test_pred, index=test.index)
    
    return results # Your Answer Here

salary_predictions()

1       0.005034
2       0.987573
5       0.987976
8       0.117562
14      0.053312
18      0.021946
27      0.028183
30      0.902317
31      0.137383
34      0.018777
37      0.020159
40      0.016472
45      0.005846
54      0.174362
55      0.538832
60      0.080264
62      0.988456
65      0.987422
77      0.008042
79      0.017314
97      0.009253
101     0.003136
103     0.562415
108     0.010781
113     0.063559
122     0.001791
141     0.166784
142     0.988456
144     0.012987
145     0.698570
          ...   
913     0.009200
914     0.010464
915     0.001552
918     0.021805
923     0.016919
926     0.028838
931     0.007891
934     0.001371
939     0.001133
944     0.001504
945     0.016919
947     0.028178
950     0.018011
951     0.004870
953     0.004007
959     0.001049
962     0.001615
963     0.048345
968     0.025535
969     0.033527
974     0.017409
984     0.001910
987     0.026094
989     0.028178
991     0.026421
992     0.001003
994     0.001504
996     0.0012

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [25]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [66]:
def new_connections_predictions():
    
    # Your Code Here
    
    from sklearn.ensemble import GradientBoostingClassifier
    
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
        
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    
    df['common_neighbors'] = df.index.map(lambda node: len(list(nx.common_neighbors(G, node[0], node[1]))))
    df['resource_allocation'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
    df['jaccard_coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['adamic_adar_index'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
    df['community_common_neighours'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, df.index)]
    df['community_resource_allocation'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, df.index)]
    
    df = future_connections.join(df, how='outer')
    
    train = df[~df['Future Connection'].isnull()]
    test = df[df['Future Connection'].isnull()]
    
    X_train = train[['preferential_attachment', 'common_neighbors', 'resource_allocation', 
                     'jaccard_coefficient', 'adamic_adar_index', 'community_common_neighours',
                     'community_resource_allocation']]
    X_test = test[['preferential_attachment', 'common_neighbors', 'resource_allocation', 
                   'jaccard_coefficient', 'adamic_adar_index', 'community_common_neighours',
                   'community_resource_allocation']]
    y_train = train['Future Connection']
    
    clf = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
    test_pred = clf.predict_proba(X_test)[:, 1]
    
    results = pd.Series(test_pred, index=test.index)

    
    return results # Your Answer Here

new_connections_predictions()

(0, 9)          0.027461
(0, 19)         0.055647
(0, 20)         0.219260
(0, 35)         0.018474
(0, 38)         0.016054
(0, 42)         0.854177
(0, 43)         0.014916
(0, 50)         0.013691
(0, 53)         0.556320
(0, 54)         0.038074
(0, 62)         0.339397
(0, 63)         0.055089
(0, 69)         0.027461
(0, 72)         0.012956
(0, 83)         0.175172
(0, 90)         0.025665
(0, 91)         0.017838
(0, 95)         0.103993
(0, 100)        0.027461
(0, 106)        0.987957
(0, 108)        0.018474
(0, 109)        0.012956
(0, 110)        0.012956
(0, 118)        0.015500
(0, 122)        0.025665
(0, 131)        0.037685
(0, 133)        0.721451
(0, 140)        0.060816
(0, 149)        0.120425
(0, 151)        0.016054
                  ...   
(988, 989)      0.012956
(988, 996)      0.012956
(988, 997)      0.042857
(988, 1000)     0.012956
(988, 1002)     0.012956
(989, 994)      0.012956
(989, 996)      0.012956
(989, 1003)     0.012956
(989, 1004)     0.012956
